In [ ]:
%matplotlib inline

import os
import io
import sys
from os.path import join as pjoin
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
#import geopandas as gpd
from datetime import datetime

from extremes import returnLevels, empReturnPeriod
from distributions import fittedPDF

from scipy.optimize import curve_fit
from scipy.stats import genpareto, scoreatpercentile

import seaborn as sns
sns.set_context("talk")
sns.set_style("whitegrid")

In [ ]:
def loadObservations(stnId):
    """
    Load the observations from file for a given BoM station, where the observations have
    been selected from the complete digital history of daily maximum wind speeds, where a cyclone has 
    passed within 200 km of the station, and the station was open at the time of passage.
    
    :param int stnId: Bureau of Meteorology Station identification number
    
    :returns: data frame containing the gust wind speed, direction and cyclone name
              based on passage of cyclones near the selected station. If no observation
              file is found (`Exception.FileNotFoundError`), return `None`
    """
    
    names = ['recid', 'stnId', 'datetime', 'gust',
             'direction', 'quality', 'cycName', 'cycId']
    
    filename = pjoin(obsPath, "bom_{0:06d}.csv".format(stnId))
    try:
        obsdf = pd.read_csv(filename, skiprows=1, names=names,
                            parse_dates=[2], infer_datetime_format=True)
    except IOError:
        print("No data file for stnId: {0}".format(stnId))
        return None
    return obsdf

def getStationDates(stnId):
    """
    Retrieve the length of observed record in years, based on the start and end dates
    of the observational data. 
    
    :param int stnId: Bureau of Meteorology Station identification number
    
    :returns: number of years between the start and end of the data on file.
    """
    startYear = stndf.loc[stnId]['stnDataStart']
    endYear = stndf.loc[stnId]['stnDataEnd']
    numYears = endYear - startYear + 1
    return numYears

STNTYPES = [('st', 'S2'), ('stnId', 'i'), ('stnDistCode', 'S4'), ('stnName', 'S'), 
            ('stnDateOpen', 'S10'), ('stnDateClosed', 'S10'), ('stnLat', 'f8'), 
            ('stnLon', 'f8'), ('method', 'S15'), ('state', 'S3'), 
            ('stnElevation', 'f8'), ('baroElev', 'i'), ('stnWMONumber', 'i'), ('stnDataStart', 'i'), 
            ('stnDataEnd', 'i'), ('blank', 'S3'), ('percentcomplete', 'f8'), ('pcqualy', 'f8'), 
            ('pcqualn', 'f8'), ('pcqualw', 'f8'), ('pcquals', 'f8'), ('pcquali', 'f8'), ('end', 'S1')]
STNCONVERT = {'stnName' : str.rstrip}

def plotObservedHazard(locId, ax):
    obsdf = loadObservations(locId)
    if obsdf is None:
        return ax
    numYears = getStationDates(locId)
    data = np.zeros(int(numYears * 365.25))
    wspd = np.sort(np.array(obsdf['gust']))*1.114 # Include conversion to 0.2 second wind gust
    data[-len(wspd):] = wspd
    emprp = empReturnPeriod(data)
    
    ax.scatter(emprp[data > 1], data[data > 1], s=50,
                color='k', marker='x', label="Empirical ARI (n={0})".format(len(wspd)))
    return ax

def calcSD(pars, cov, rate, npyr, intervals):
    nsims = 1000
    rps = np.zeros((nsims, len(intervals)))
    for i in range(nsims):
        xi = pars[0] + np.random.normal(0, np.sqrt(np.abs(cov[0,0])))
        mu = pars[1] + np.random.normal(0, np.sqrt(np.abs(cov[1,1])))
        sig = pars[2] + np.random.normal(0, np.sqrt(np.abs(cov[2,2])))
        rps[i, : ] = mu + (sig / xi) * (np.power(intervals * npyr * rate, xi) - 1.)
    lowerrp = scoreatpercentile(rps, 5, axis=0)
    upperrp = scoreatpercentile(rps, 95, axis=0)
    return lowerrp, upperrp

In [ ]:
obsPath = "C:/WorkSpace/data/derived/tcobs/daily"
stationFilePath = "C:/WorkSpace/data/raw/daily_max_wind_gust/"
stnfile = pjoin(stationFilePath, "DC02D_StnDet_999999999425050.txt")

stndf = pd.read_csv(stnfile, parse_dates=[4, 5],
                        usecols=(1,2,3,4,5,6,7,9,10,12,13,14,16), 
                        names = np.dtype(STNTYPES).names,
                        skiprows=1, engine='python', index_col='stnId', 
                        converters=STNCONVERT)
stationNameList = list(stndf['stnName'])

In [ ]:
stnId = stndf.index[stndf['stnName'] == 'CAIRNS AERO'].tolist()[0]
obsdf = loadObservations(stnId)
wspd = obsdf['gust'].values*1.114


In [ ]:
bins = np.arange(0.5, 100, 1)
n, bins = np.histogram(wspd, bins, normed=True)
centres = 0.5*(bins[1:]+bins[:-1])

pars,cov = curve_fit(lambda x, xi, mu, sig: genpareto.pdf(x, xi, loc=mu, scale=sig), 
                     centres, n, p0=[0, np.mean(wspd), np.mean(wspd)], maxfev=10000 )

gpd = genpareto.fit(wspd)


In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
plotObservedHazard(stnId, ax)
ax.set_xscale('log')
ax.set_xlim((1,100))
ax.set_ylim((0, 100))
ax.grid('on', which='both')
intervals = bins
npyr = 365.25
fxi, fmu, fsig = gpd
cxi, cmu, csig = pars
numYears = getStationDates(stnId)
rate = len(wspd)/(npyr*numYears)
frp = fmu + (fsig / fxi) * (np.power(intervals * npyr * rate, fxi) - 1.)
crp = cmu + (csig / cxi) * (np.power(intervals * npyr * rate, cxi) - 1.)
lrp, urp = calcSD(pars, cov, rate, npyr, intervals)

ax.semilogx(intervals, frp, 'r', label="MLE fit")
ax.semilogx(intervals, crp, 'k', label="LM fit")

ax.semilogx(intervals, lrp, '0.5')
ax.semilogx(intervals, urp, '0.5', label='90% CI on LM')
ax.legend()

In [ ]:
import lmfit
x = np.linspace(1, 100, 100)
y = wspd
p = lmfit.Parameters()
p.add_many(('xi', pars[0], True, -np.inf, 2.), ('mu', pars[1], True, 0., np.inf),('sig', pars[2], True, 0, np.inf))

def residual(p, x, y):
    return genpareto.pdf(x, p['xi'], loc=p['mu'], scale=p['sig']) - y


In [ ]:
mini = lmfit.Minimizer(residual, p, fcn_args=(centres, n), nan_policy='omit')

# first solve with Nelder-Mead
out1 = mini.minimize(method='Nelder')
out2 = mini.minimize(method='leastsq', params=out1.params)
lmfit.report_fit(out2.params, min_correl=0.25)

In [ ]:
cmu = out2.params['mu'].value
cxi = out2.params['xi'].value
csig = out2.params['sig'].value
rate = len(obsdf['gust'].values)/(npyr*numYears)
crp = cmu + (csig / cxi) * (np.power(intervals * npyr * rate, cxi) - 1.)
fig, ax = plt.subplots(figsize=(12,8))
plotObservedHazard(stnId, ax)
intervals = bins

ax.semilogx(intervals, crp, 'r', label="LM fit")
lrp, urp = calcSD((cxi, cmu, csig), out2.covar, rate, npyr, intervals)
ax.semilogx(intervals, lrp, '0.5')
ax.semilogx(intervals, urp, '0.5', label='90% CI on LM')
ax.set_xlim((1, 100))
ax.set_ylim((0, 100))
ax.grid('on',which='both')
ax.set_xlabel("ARI (years)")
ax.set_ylabel("Wind speed (m/s)")
ax.legend()
plt.savefig(pjoin("C:/WorkSpace/data/derived/tcobs/daily", "{0:06d}_obsfit.png".format(stnId)))

In [ ]:
sns.distplot(wspd, bins = np.arange(1, 100.,))
gpdpdf = genpareto.pdf(bins, cxi, loc=cmu, scale=csig)
plt.plot(bins, gpdpdf)

In [ ]:
print(lmfit.fit_report(out2))

In [ ]:
ci, trace = lmfit.conf_interval(mini, out2, sigmas=[1, 2],
                                trace=True, verbose=False, maxiter=1000)
lmfit.printfuncs.report_ci(ci)

In [ ]:
x, y, prob = trace['mu']['mu'], trace['mu']['xi'], trace['mu']['prob']
x2, y2, prob2 = trace['xi']['xi'], trace['xi']['mu'], trace['xi']['prob']
plt.scatter(x, y, c=prob ,s=30)
plt.colorbar()
plt.xlabel(r'$\mu$')
plt.ylabel(r'$\xi$')

In [ ]:
plot_type = 2
if plot_type == 0:
    plt.plot(x, y)
    plt.plot(x, residual(out2.params) + y)

elif plot_type == 1:
    cx, cy, grid = lmfit.conf_interval2d(mini, out2, 'mu', 'xi', 50, 50)
    plt.contourf(cx, cy, grid, np.linspace(0, 1, 11))
    plt.xlabel('mu')
    plt.colorbar()
    plt.ylabel('xi')

elif plot_type == 2:
    cx, cy, grid = lmfit.conf_interval2d(mini, out2, 'mu', 'xi', 50, 50)
    plt.contourf(cx, cy, grid, np.linspace(-1, 1, 11))
    plt.xlabel('mu')
    plt.colorbar()
    plt.ylabel('xi')

elif plot_type == 3:
    cx1, cy1, prob = trace['a1']['a1'], trace['a1']['t2'], trace['a1']['prob']
    cx2, cy2, prob2 = trace['t2']['t2'], trace['t2']['a1'], trace['t2']['prob']
    plt.scatter(cx1, cy1, c=prob, s=30)
    plt.scatter(cx2, cy2, c=prob2, s=30)
    plt.gca().set_xlim((2.5, 3.5))
    plt.gca().set_ylim((11, 13))
    plt.xlabel('a1')
    plt.ylabel('t2')

if plot_type > 0:
    plt.show()